<center><h1> Determination of the best chromatography type </h1></center>
    
## **Introduction**

<p style="text-align: justify;"> Chromatography is a crucial analytical method in chemistry used for separating, identifying, and quantifying components in a mixture. Selecting the appropriate type of chromatography (e.g., gas chromatography, liquid chromatography, ion chromatography, size exclusion chromatography) is essential for optimal results. This is why we decided to <b>create a programming tool that allows to determine easily the chromatpgraphy type as well as the best eluent and conditions needed to carry out the chromatography</b>. <br>
This report details the development and execution of a programming project aimed at creating a tool to determine the most suitable chromatography technique for analyzing a given mixture of molecules. The project was conducted using python 3.10. The <b> different functions created to make this tool are listed </b> in this notebook. <br>
To carry out this project, PubChem has been used as a data base in order to extract all the important information that were needed to establish the best chromatography type (boiling_point, pka, LogP, Molar Weight). Then, a function that takes into account all the different information contained in a dataframe has been created and finaly, a user interface has been created.

## **Objectives**

The primary objective of this project was to develop a program that:

- Determine physicochemical properties from PubChem input data on the molecular composition of a mixture.
- Store the properties
- Recommends the most suitable chromatography technique based on predefined criteria and standards.

## **References**

- Pubchemprops: <b>Github repo of mawansui </b> (https://github.com/mawansui/pubchemprops) used to retrieve most of the properties of the molecules from Pubchem
- pka_lookup.py: <b>Github repo of Khoivan88 </b> (https://github.com/khoivan88/pka_lookup) which request the needed dictionnary of pka on PubChem.
 

## **Methodology**

### 1. Graphical User Interface ###

<p style="text-align: justify;">A simple user interface was developed using Tkinter, allowing users to input the name of several molecules and obtain chromatography recommendations. The user can give molecule names that will be added to a listbox to keep in mind the different molecules in the mixture. The user then decides to determine the best chromatography type that will be indicated on the interface using the "Determine Chromatography" button. If an error occurs, an error message will be displayed using a message box on the user interface.

<div style="display: flex; justify-content: center; align-items: center; padding: 20px;">
    <div style="margin: 10px;">
        <img src="../assets/edited-hydrocarbure.gif" alt="hydrocarbures" style="width: 400px; height: auto;">
    </div>
    <div style="margin: 10px;">
        <img src="../assets/edited-naphtalene-.gif" alt="HPLCexample" style="width: 400px; height: auto;">
    </div>
</div>


These two examples show how the results appear on the user ineterface when molecules are added. Above, two different mixtures are given, hence, two different results appear on the screen.


### 2. Data Collection and Preparation ###
<p style="text-align: justify;">
A dataset containing various molecular properties (e.g., molecular weight, polarity, solubility) and corresponding successful chromatography types was compiled.
Data preprocessing included handling missing values such as the pKa values. Relevant features such as molecular weight, polarity index, boiling point, pKa and solubility were selected and added to a data frame. 

First we import the function from the Chrfinder.py file.

In [1]:
from Chrfinder import find_pka, find_boiling_point, get_df_properties

Then we can create a list of compounds, which will normally be done by tkinter.

In [2]:
mixture = ["caffeine", "Aspirin", "Maleic acid"]

Then we run and print the dataframe:

In [3]:
print(get_df_properties(mixture))

      CID MolecularFormula  MolecularWeight                     InChIKey  \
0    2519        C8H10N4O2           194.19  RYYVLZVUVIJVGH-UHFFFAOYSA-N   
1    2244           C9H8O4           180.16  BSYNRYMUTXBXSQ-UHFFFAOYSA-N   
2  444266           C4H4O4           116.07  VZCYOOQTPOCHFL-UPHRSURJSA-N   

                         IUPACName  XLogP    pKa  Boiling Point  
0  1,3,7-trimethylpurine-2,6-dione   -0.1  14.00         177.93  
1          2-acetyloxybenzoic acid    1.2   3.47         140.00  
2          (Z)-but-2-enedioic acid   -0.3   1.83         135.00  
      CID MolecularFormula  MolecularWeight                     InChIKey  \
0    2519        C8H10N4O2           194.19  RYYVLZVUVIJVGH-UHFFFAOYSA-N   
1    2244           C9H8O4           180.16  BSYNRYMUTXBXSQ-UHFFFAOYSA-N   
2  444266           C4H4O4           116.07  VZCYOOQTPOCHFL-UPHRSURJSA-N   

                         IUPACName  XLogP    pKa  Boiling Point  
0  1,3,7-trimethylpurine-2,6-dione   -0.1  14.00         177

### 3. Determination of the chromatography type ###
<p style="text-align: justify;">
A function has been created to determine the most suitable chromatography type based on the physical-chemical properties of the molecules saved on a data frame. To do this the following dependencies has tp be installed:

In [4]:
import sys
import os
notebook_path = os.getcwd()  # Get the current working directory
src_path = os.path.abspath(os.path.join(notebook_path, "../src/Chrfinder"))
sys.path.insert(0, src_path)

import pandas as pd

Pandas will create a data frame from a dictionary as follows:

In [5]:
data = {
    'Mixture': ['Caf', 'Ace', 'Asp'],
    'Boiling Point': [178, 332.7, 246],
    'XLogP': [-0.07, -1.33, 0.07],
    'pKa': [[14], [3.02], [1.08, 9.13]],
    'MolecularWeight': [194, 204, 133],
}

df = pd.DataFrame(data)
print(df)

  Mixture  Boiling Point  XLogP           pKa  MolecularWeight
0     Caf          178.0  -0.07          [14]              194
1     Ace          332.7  -1.33        [3.02]              204
2     Asp          246.0   0.07  [1.08, 9.13]              133


As can be seen in the second column it is indicated the names of all the molecules from the mixture. Then the properties used by the function are the boiling point in [°C], the XlogP, the pKa and the molecular weight in [g/mol].

Following an example of the functionality of the function:

In [6]:
from Chrfinder import det_chromato
Mixture_chromato_type, eluent_nature, proposed_pH = det_chromato(df)

The function will look up column per column in the data frame and search the maximum or the minimum of each fisical or chemical properties. Based on that it start to predict the best chromatogrphy to perform:

The GC will be performed if the max **boiling temperature** of the mixture is under 250 °C, this will ensure the high volatility of the mixture.

The **molar weight** will indicate if the steric exclusion chromatography is allowed or not. It is suitable for heavy molecules, thus for the compounds in the mixture with a mass larger than 2000 g/mol.

The choice between high pressure liquid chromatography (HPLC) and ionic chromatography (IC) is determined by the **XlogP**, wich is an indicator of the hydrophilicity of the molecules. Negative values for the **XlogP** indicates that the molecule is likely to dissolve in water (thus hydrophile), positive values indicates that the molecule is more likely to dissolve in organic solvents (thus hydrofobe). Therefore the more hydrophile is the molecule the more likely it will ionize, hence the suitable chromatography will be IC. 

In the liquid chromatography such as HPLC or IC the pH of the eluent is an important parameter which can lead to a good separation. All the molecules have to be stable at this pH otherwise they can degrade and the separation will not be good. to ensure the stability of the molecules, the function propose a pH wich is 2 values above the minimum **pKa** of the mixture compunds.

In [7]:
print(f"The advisable chromatography type is: {Mixture_chromato_type}")
print(f"Eluent nature: {eluent_nature}")
if proposed_pH is not None:
    print(f"Proposed pH for the eluent: {proposed_pH}")

The advisable chromatography type is: HPLC on reverse stationary phase using C18 column
Eluent nature: organic or hydro-organic
Proposed pH for the eluent: 3.08


### 4. Explanation of each part of get_df_properties(mixture) (c.f. Methodology.2.) ###
#### Getting 'CID', 'MolecularFormula', 'MolecularWeight', 'InChIKey', 'IUPACName', 'XLogP' using PubChem requests 
(only missing Boiling Point and pka)

This part of the code uses pubchempy and the code of *Maxim Shevelev* (pubchemprops.py) to easily find most of the properties. It takes into arguments a list of compound's names. It will return a dictionnary for each compound with the following properties: 'CID', 'MolecularFormula', 'MolecularWeight', 'InChIKey', 'IUPACName', 'XLogP'.

The tkinter interface returns a list of compounds as string. The first part of the code iterates through the whole list (compound_list), and encodes every compound name (string) as URL. Then it will search for the properties on pubchem using pubchempy and add them into a dictionnary.

In [8]:
from Chrfinder.Chrfinder import get_df_properties

compound_list = ["caffeine", "Aspartame", "Acesulfame K"]
#Delete '#' for a list with a wrong name
#compound_list = ["Water", "Acetone", "Wrong name"]

get_df_properties(compound_list, verbose = True)

{'CID': 2519, 'MolecularFormula': 'C8H10N4O2', 'MolecularWeight': 194.19, 'InChIKey': 'RYYVLZVUVIJVGH-UHFFFAOYSA-N', 'IUPACName': '1,3,7-trimethylpurine-2,6-dione', 'XLogP': -0.1}
{'CID': 134601, 'MolecularFormula': 'C14H18N2O5', 'MolecularWeight': 294.3, 'InChIKey': 'IAOZJIPTCAWIRG-QWRGUYRKSA-N', 'IUPACName': '(3S)-3-amino-4-[[(2S)-1-methoxy-1-oxo-3-phenylpropan-2-yl]amino]-4-oxobutanoic acid', 'XLogP': -2.7}
{'CID': 11074431, 'MolecularFormula': 'C4H4KNO4S', 'MolecularWeight': 201.24, 'InChIKey': 'WBZFUFAFFUEMEI-UHFFFAOYSA-M', 'IUPACName': 'potassium;6-methyl-2,2-dioxo-1-oxa-2lambda6-thia-3-azanidacyclohex-5-en-4-one', 'XLogP': None}
        CID MolecularFormula  MolecularWeight                     InChIKey  \
0      2519        C8H10N4O2           194.19  RYYVLZVUVIJVGH-UHFFFAOYSA-N   
1    134601       C14H18N2O5           294.30  IAOZJIPTCAWIRG-QWRGUYRKSA-N   
2  11074431        C4H4KNO4S           201.24  WBZFUFAFFUEMEI-UHFFFAOYSA-M   

                                           

,CID,MolecularFormula,MolecularWeight,InChIKey,IUPACName,XLogP,pKa,Boiling Point
0,2519,C8H10N4O2,194.19,RYYVLZVUVIJVGH-UHFFFAOYSA-N,"1,3,7-trimethylpurine-2,6-dione",-0.1,14.00,177.93
1,134601,C14H18N2O5,294.30,IAOZJIPTCAWIRG-QWRGUYRKSA-N,(3S)-3-amino-4-[[(2S)-1-methoxy-1-oxo-3-phenyl...,-2.7,3.19,NaN
2,11074431,C4H4KNO4S,201.24,WBZFUFAFFUEMEI-UHFFFAOYSA-M,"potassium;6-methyl-2,2-dioxo-1-oxa-2lambda6-th...",NaN,NaN,NaN


#### <ins> Finding pka using Pubchem from InchiKey String</ins>

Using the InchiKey String found by the function right before, the following function **returns the first pka found on PubChem as string**. This value, similary to the Boiling temperature, is a lot harder to find. This script uses a file of *Khoi Van* named **pka_lookup.py** which request the needed dictionnary of strings on PubChem. This means it takes **quite a while to find the string**, but creating a database is in scope. 

From the string found, this code extracts the pka_value from the dictionnary and returns the value as a string which will be converted in float in the function of Chrfinder.py.

In [9]:
from Chrfinder.Chrfinder import find_pka

#inchikey of caffeine
inchikey_string = 'RYYVLZVUVIJVGH-UHFFFAOYSA-N'

find_pka(inchikey_string, verbose=True)

{'source': 'Pubchem', 'Pubchem_CID': '2519', 'pKa': '14', 'reference': 'https://www.sigmaaldrich.com/content/dam/sigma-aldrich/docs/Sigma-Aldrich/Product_Information_Sheet/c0750pis.pdf', 'Substance_CASRN': '58-08-2', 'Canonical_SMILES': 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C', 'Isomeric_SMILES': 'CN1C=NC2=C1C(=O)N(C(=O)N2C)C', 'InChI': 'InChI=1S/C8H10N4O2/c1-10-4-9-6-5(10)7(13)12(3)8(14)11(6)2/h4H,1-3H3', 'InChIKey': 'RYYVLZVUVIJVGH-UHFFFAOYSA-N', 'IUPAC_Name': '1,3,7-trimethylpurine-2,6-dione'}


'14'

#### <ins> Finding Boiling Temperature using Pubchem from name</ins>

Using the names of compound_list, the following function **returns the mean of the celsius and Fahrenheit Boiling Temperatures found on PubChem as float**. This value, similary to the Boiling temperature, is a lot harder to find. This script uses a file of *Maxim Shevelev* named **pubchemprops.py** which request the needed dictionnary of strings on PubChem. This means it takes **quite a while to find the string**, but creating a database is in scope. 

From the string_value in text_dict (extracted from PubChem), this code extracts all the boiling points from the dictionnary and returns the mean after converting Fahrenheit in celsius. The output is a float with 2 decimals.

In [10]:
from Chrfinder.Chrfinder import find_boiling_point

mixture = "Caffeine"
find_boiling_point(mixture, verbose= True)

{'Boiling Point': [{'ReferenceNumber': 2, 'Reference': ['National Toxicology Program, Institute of Environmental Health Sciences, National Institutes of Health (NTP). 1992. National Toxicology Program Chemical Repository Database. Research Triangle Park, North Carolina.'], 'Value': {'StringWithMarkup': [{'String': '352 °F at 760 mmHg (sublimes) (NTP, 1992)'}]}}, {'ReferenceNumber': 24, 'Reference': ['http://www.inchem.org/documents/icsc/icsc/eics0405.htm'], 'Value': {'Number': [178]}}, {'ReferenceNumber': 51, 'Description': 'PEER REVIEWED', 'Reference': ["O'Neil, M.J. (ed.). The Merck Index - An Encyclopedia of Chemicals, Drugs, and Biologicals. Cambridge, UK:  Royal Society of Chemistry, 2013., p. 289"], 'Value': {'StringWithMarkup': [{'String': '178 °C (sublimes)'}]}}, {'ReferenceNumber': 71, 'Value': {'StringWithMarkup': [{'String': '178 °C (sublimes)'}]}}], 'Vapor Pressure': [{'ReferenceNumber': 49, 'Value': {'StringWithMarkup': [{'String': '0.00000001 [mmHg]'}]}}, {'ReferenceNumbe

177.93

## **Results** ##
<p style="text-align: justify;">
The final model demonstrated high accuracy in recommending the appropriate chromatography technique based on the molecular properties of the mixture. Key results include:

**Feature Importance:** 
<p style="text-align: justify;">
Polarity index, molecular weight, and solubility were identified as the most influential features in determining the suitable chromatography type.

**User Feedback:**
<p style="text-align: justify;">
The interactive nature of the user interface allowed for seamless user input and clear presentation of results, enhancing the user experience.
    
When error occurs, messages appear to inform the user of the encountered error. An example of this feature is shown bellow:

<div style="text-align: center; padding: 20px;">
    <img src="../assets/edited-error-molecule-not-added.gif" alt="errornomolecule" style="width: 400px; height: auto;">
</div>

  

## **Limits** ##

The example bellow shows the limit of the program. Indeed, the suitable chromatography given by the interface is not correct, this is due to the fact that the boiling point is not given in pubchem which generates an error in the determination of the chromatography. Hence, a lack of data is limiting our project.

<div style="text-align: center; padding: 20px;">
    <img src="../assets/edited-limit.gif" alt="limit" style="width: 400px; height: auto;">
</div>

##### Take into account: 
- It takes into account **wrong names** and **compounds with no pages on PubChem**: it returns "The_compound_name" not found on PubChem'
- The code works even with missing boiling point or pka in dataframe.

##### To improve: 
- **Build a database**: mostly with thermostability for better choice of chromatography;
- Taking into account **multiple pKa** values for polyacids for exemple;
- Optimize the research: search only one time the **same name**, don't search for **one name**;
- **Easier usage and options** of functionnalities to search physicalchemical properties in a dataframe;



## **Encountered difficulties** ##

A lot of challenges were faced to make request to pubchem:
- Handling of **errors concerning pubchem**, no page found, wrong compound name, etc
  
- How to **extract string using regex**, used for pka, boiling temperature (°C) and (°F).
  
- Handling of **pka errors wrongly written in Pubchem** (either value in value, either input in the key of the dictionnary) <br>
    {"pka = 20.0", } instead of {'pKa': '14'}
  
- Tests of dataframe object (pd.testing.assert_frame_equal(df, expected_df)) and Tk() listbox
   
## **Conclusion** ##
<p style="text-align: justify;">
The project successfully developed a robust program for recommending chromatography techniques based on molecular composition. The use of a user interface facilitated an efficient and interactive development process, enabling easy experimentation and visualization. This tool has significant potential applications in chemical analysis, improving efficiency and accuracy in selecting the appropriate analytical methods. The main goal is to generate a database with commun eluants and their Boiling_point, pka, thermal stability. The last property needs to be researched in laboratory.

## **Future Work** ##

Future enhancements could include:

- Expanding the dataset to cover more diverse molecular structures and additional chromatography techniques.
- Improve the code to have a more efficient way to take the information on the molecules contained in the mixture to have the results more rapidly.
- Integrating the program with real-time data acquisition systems for automated analysis.
- Enhancing the user interface with more sophisticated visualization tools and user-friendly features.
- By continuing to refine and expand this tool, it can establish itself as a valuable resource for chemists and researchers in analytical laboratories.

This report provides an overview of the programming project conducted using Python, detailing the development process, results, and future directions for the chromatography recommendation program.
